In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import cyipopt
from pybullet_suite import *
from panda_collision import *
from signed_distance import *
from pathlib import Path

pybullet build time: Dec 14 2022 00:46:04


In [2]:
mesh_path = Path(PANDA_URDF).parent/"meshes/collision"
link_names = link_names = [
    'link0', 'link1', 'link2', 'link3', 
    'link4', 'link5', 'link6', 'link7', 'hand', 
    'finger', 'finger'
]

In [3]:
fk = get_fk_fn(PANDA_URDF)
pc = get_pointclouds(mesh_path, link_names)

Robot name: panda


concatenating texture: may result in visual artifacts
only got 79/100 samples!


In [6]:
offset = SE3.from_translation(jnp.array([0., 0, -1]))
half_extents = jnp.array([10., 10, 1])
ground = box(offset, half_extents)
circ = circle(jnp.array([0.5,0,0.5]), 0.2)
env = EnvSDF((circ, ground))

In [7]:
def robot_penet(q):
    tfs = fk(q)
    points = pc.apply_transforms(tfs)
    return env.penetration_sum(points, pc.num_points, 0.02)

In [10]:
%timeit robot_penet(jnp.zeros(7))

58.1 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
pc.apply_transforms(fk(jnp.zeros(7)))

Array([[ 6.3396730e-02, -1.3823391e-02,  5.6593366e-02],
       [-1.4391968e-01, -7.1363188e-03,  2.4019109e-02],
       [-1.3907564e-01,  4.7267664e-02,  1.7710578e-07],
       ...,
       [ 6.0018450e-02,  2.9043026e-02,  8.3249056e-01],
       [ 6.5231413e-02,  3.3720437e-02,  8.2052982e-01],
       [ 4.5467369e-02,  4.7129169e-02,  8.4266722e-01]], dtype=float32)